In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl

def prepare_dataframe(infile, filters={'test_pos': 2}, renaming=None):
    '''
    Reads a dataframe, filters it and puts human-friendly names in some default entries
    plus additional ones specified by the renaming parameter: dict(original->new_name)
    '''
    df = pd.read_csv(infile)
    df = df.rename(columns = {'strat':'portfolio'}) # renames the portfolio column

    # display(df.describe())
    
    # human-friendly portfolio names
    df.replace({'WR_LR_RR_HR': 'rush', 'WR_LR_RR_HR_BB_BK': 'rush+suport', 'WR_LR_RR_HR_WD_LD_RD_HD': 'rush+defense',  'WR_LR_RR_HR_WD_LD_RD_HD_BB_BK': 'all'}, inplace=True)
    
    # human-friendly feature selectors
    df.replace({'materialdistancehp': 'Simple', 'quadrantmodel': 'Per-quadrant'}, inplace=True)
    
    #if there is any other rename, do it:
    if renaming is not None:
        df.replace(renaming, inplace=True)
        
    return filter_data(df,filters)

def filter_data(data, filters):
    '''
    Excludes data from the dataframe according to user criteria,
    passed as a dict(colname -> value)
    '''
    for fname,fvalue in filters.items():
        if isinstance(fvalue, float):
            #print(fname, fvalue, df[fname])
            data = data[np.isclose(data[fname],fvalue)] 
        else:
            data = data[data[fname] == fvalue] 
        # drops the column
        data.drop(fname, axis=1, inplace=True)
    return data


def map_feature_portfolio(infile, metric='%score', 
                          mean_across_maps=False, filters={'test_pos': 2}, 
                          unstack_level = 2, renaming=None):
    
    df = prepare_dataframe(infile, filters, renaming)
    
    # the columns I want to see
    #indexes = ['mapname', 'feature', 'portfolio', 'alpha', '_lambda', 'train_matches', 'dec_int', 'test_pos']
    indexes = ['train_matches', 'mapname', 'feature', 'portfolio', 'alpha', '_lambda', 'dec_int', 'test_pos']
    
    # removes filter keys from the indexes, as they're no longer part of the data frame
    for fname,fvalue in filters.items():
        indexes.remove(fname)
        
    # display(df.describe())
    
    # builds a hierarchical (multiindex) data frame (kinda groups by map, strategy & units)
    hier = df.set_index(indexes) 
    #hier.index = natsorted(hier.index)
    #print(np.where(hier.index.duplicated()))  # finds entries with duplicate indexes, thx: https://stackoverflow.com/a/28652153/1251716
    
    # unstack the hierarchical dataframe up to the portfolio (all other parameters are grouped?)
    unstacked = hier.unstack(unstack_level)[metric]
    
    if mean_across_maps: 
        # if i group by the 'feature' level, i can get the mean on all maps played using that feature extractor
        return unstacked.groupby(level=['feature']).mean()
    else:
        return unstacked
        
        
def autolabel(ax):
    """
    Attach a text label above each bar displaying its height 
    (source: https://stackoverflow.com/a/22689127/1251716)
    """
    rects = [rect for rect in ax.get_children() if isinstance(rect, mpl.patches.Rectangle)]
    for rect in rects:
        height = rect.get_height()
        if height != 1: #little hack to avoid a dummy 1 between bars
            ax.text(rect.get_x() + rect.get_width()/2., 1.001*height,
                    '%d' % int(height),
                    ha='center', va='bottom')

In [ ]:
# displays the results for all adversaries
opponents = ['A3N', 'SSSmRTS','StrategyTactics','PuppetSearchMCTS','WorkerRush','LightRush'] 

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100, 'train_matches': 100000, '_lambda': 0.5, 'alpha': 0.01,}

for o in opponents:
    print("---{}, no search budget---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_avg.csv'.format(o), '%score', True, filters))
    
    print("---{}, search budget=100ms---".format(o))
    display(map_feature_portfolio('../results/parameters/{}_b100_avg.csv'.format(o), '%score', True, filters))

In [ ]:
### ONLY LIGHT & WORKER RUSH WITH b0 and b100:
# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all'}

for o in ['WorkerRush', 'LightRush']:
    for b in [0, 100]:
        print("---{}, search budget={}ms---".format(o, b))
        display(map_feature_portfolio('../results/parameters/{}_b{}_avg.csv'.format(o,b), '%score', False, filters))

In [ ]:
### This is to check the feature extractor on regular maps
import matplotlib.pyplot as plt

print('# Comparing feature extractors on "regular" maps')
# filters by a specific set of parameters
#filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all',}# 'train_matches': 100000}

#for o in ['WorkerRush', 'LightRush']:
#    print("---{}---".format(o))
#    display(map_feature_portfolio('../results/parameters/{}_b0_avg.csv'.format(o), '%score', False, filters, 1))
    
#print('# Comparing feature extractors, now only at the end of training ')

filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'train_matches': 100000}
for o in ['WorkerRush', 'LightRush']:
    #print("---{}---".format(o))
    data = map_feature_portfolio('../results/parameters/{}_b0_avg.csv'.format(o), '%score', False, filters, 1)
    display(data)
    data.sort_values(by=['mapname'], ascending=False, inplace=True) # so that 8x8 comes before 32x32
    ax = data.plot(title=o, ylim=[0, 105],kind='bar')
    ax.set(xlabel="Map", ylabel="Score (%)")
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    ax.get_figure().savefig(f'/tmp/feature-vs_{o}_regular.pdf')

In [ ]:
### This is to check the feature extractor on irregular/split maps

print('# Comparing feature extractors on "irregular/split" maps')

# filters by a specific set of parameters
#filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all',}# 'train_matches': 100000}

#for o in ['WorkerRush', 'LightRush']:
#     print("---{}---".format(o))
#     display(map_feature_portfolio('../results/features/{}_b0_avg.csv'.format(o), '%score', False, filters, 1))
    
# print('# Comparing feature extractors, now only at the end of training ')

filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'train_matches': 100000}
map_rename = {'FourBasesWorkers8x8': '8x8', 'NoWhereToRun9x8': '9x8', 'TwoBasesBarracks16x16': '16x16'}

for o in ['WorkerRush', 'LightRush']:
    data = map_feature_portfolio('../results/features/{}_b0_avg.csv'.format(o), filters=filters, unstack_level= 1, renaming=map_rename)
    #print("---{}---".format(o))
    #display(data)
    data = data.loc[['8x8', '9x8', '16x16']] # to sort the columns for the barplot (https://stackoverflow.com/a/20555406/1251716)
    ax = data.plot(title=o, ylim=[0, 105],kind='bar')
    ax.set(xlabel="Map", ylabel="Score (%)")
    ax.set_xticklabels( ax.get_xticklabels(), rotation=0) # rotates the labels without changing them
    autolabel(ax)
    ax.get_figure().savefig(f'/tmp/feature-vs_{o}_irregular.pdf')

In [ ]:
'''
This cell is to plot line charts with %score according to search budget, 
train matches, map and opponent
'''

import matplotlib.pyplot as plt

# filters by a specific set of parameters
filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'feature': 'Simple'}

data_array = []
for o in ['WorkerRush', 'LightRush']:
    opp_data = []
    print(f'---{o}---')
    for b in [0, 100]:
        data = prepare_dataframe(f'../results/parameters/{o}_b{b}_avg.csv', filters=filters)
        
        # augments the dataset with the budget column
        data['budget'] = [b] * len(data)
        #display(data)
        opp_data.append(data)
    data_array += opp_data
    print(f'len(opp-data): {len(opp_data)}')
    
    df = pd.concat(opp_data) # creates a dataframe with the opponent data on both maps, it is processed below
    # here I have data for a test opponent with all desired search budgets, will try plotting per map
    #display(data)
    for mapname in ['basesWorkers8x8', 'basesWorkers32x32A']:
        print(f'\t---{mapname}----')
        # get data from a single map & remove the map column
        per_map = df[df['mapname'] == mapname] 
        per_map.drop('mapname', axis=1, inplace=True)
        #display(per_map)
        hier = per_map.set_index(['train_matches', 'budget']) 
        display(hier)
        print(np.where(hier.index.duplicated())) # prints where hierarchical has duplicate entries (cannot unstack)
        unstacked = hier.unstack(0)['%score'] # groups(?) results via score
        #display(unstacked)
        ax = unstacked.plot(title=f'{o} - {mapname}', ylim=[-1, 105], xlim=[-1, 101],style=['*-','o-','+-','s-'])
        ax.set(xticks=[0,100]) # if there's data on other budgets, put here
        ax.set(xlabel="Search budget (ms)", ylabel="Score (%)")
        ax.get_figure().savefig(f'/tmp/budget-{mapname}_vs_{o}.pdf')

'''
all_data = pd.concat(data_array) #now I have data from all opponents, search budgets & maps
#display(all_data) 
fig, axes = plt.subplots(nrows=1,ncols=2, figsize=(6,4), sharex=True)
# filters out by each map
for mapname in ['basesWorkers8x8', 'basesWorkers32x32A']:
    print(f'\t---{mapname}----')
    # get data from a single map & remove the map column
    per_map = all_data[all_data['mapname'] == mapname] 
    per_map.drop('mapname', axis=1, inplace=True)
    
    # loop over opponents
    for i,o in enumerate(['WorkerRush', 'LightRush']):
        per_map_opp = per_map[per_map["test_opp"] == o]
        axes[i].plot(per_map_opp["budget"], per_map_opp["%score"], label=o)
    #axes.margins(x=0)
        axes[i].set_title(o)
'''
    
#     display(per_map)

#     # builds a hierarchical (multiindex) data frame 
#     hier = per_map.set_index(['train_matches', 'budget', 'test_opp']) 
#     unstacked = hier.unstack(2)['%score'] # groups(?) results via score
#     wr_budget = unstacked.groupby('budget')
#     display(unstacked)
#     unstacked.plot()
pass

In [ ]:
'''
This cell is to create tables with results vs state of the art algorithms
'''

filters = {'test_pos': 2, 'dec_int': 100,  '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'feature': 'Simple'}

opponents = ['A3N', 'LightRush', 'PuppetSearchMCTS', 
             'SSSmRTS', 'StrategyTactics', 'WorkerRush']


# considers only b=100
data_array = [prepare_dataframe(f'../results/parameters/{o}_b100_avg.csv', filters=filters) for o in opponents]
df = pd.concat(data_array)
#display(df)

# regular maps:
for mapname in ['basesWorkers8x8', 'basesWorkers16x16A', 'basesWorkers32x32A']:
    # filters by map & drops the map column
    per_map = df[df['mapname'] == mapname] 
    per_map.drop('mapname', axis=1, inplace=True)
    
    print(f'---- {mapname} ----')
    display(per_map)
    
    hier = per_map.set_index(['test_opp']) 
    unstacked = hier.unstack(0)['%score']
    display(unstacked)

#the code below is to verify which opponents were not tested within the given parameters
# for o in opponents:
#     print(f'--- {o} ---')
#     display(prepare_dataframe(
#         f'../results/parameters/{o}_b100_avg.csv', 
#         filters={'test_pos': 2, 'dec_int': 100, '_lambda': 0.5, 'alpha': 0.01, 'portfolio': 'all', 'feature': 'Simple'}
#     ))
